In [29]:
from darli.robots import RobotModel
from robot_descriptions import ur5_description
import casadi as cs
model = RobotModel(ur5_description.URDF_PATH)

In [30]:
nq = model.nq # dimensionality of configuration  
nv = model.nv # dimensionality of generilized velocities
nu = model.nu # dimensionality  of control inputs
q_min, q_max = model.q_min, model.q_max # minimal and maximal limits on joint pos
nq, nv, nu

(6, 6, 6)

In [31]:
nb = len(model._model._kindyn.joint_names()) - 1

In [34]:
parameters = cs.SX.sym('p', nb*10)

In [35]:
parameters

SX([p_0, p_1, p_2, p_3, p_4, p_5, p_6, p_7, p_8, p_9, p_10, p_11, p_12, p_13, p_14, p_15, p_16, p_17, p_18, p_19, p_20, p_21, p_22, p_23, p_24, p_25, p_26, p_27, p_28, p_29, p_30, p_31, p_32, p_33, p_34, p_35, p_36, p_37, p_38, p_39, p_40, p_41, p_42, p_43, p_44, p_45, p_46, p_47, p_48, p_49, p_50, p_51, p_52, p_53, p_54, p_55, p_56, p_57, p_58, p_59])

In [38]:
joint_regressor = model._model._kindyn.jointTorqueRegressor()

In [40]:
tau = joint_regressor(model._q, model._v, model._dv) @ parameters

In [41]:
tau

SX(@1=0.13585, @2=4.89664e-12, @3=cos(q_1), @4=sin(q_1), @5=((@2*@3)-@4), @6=((@2*@4)+@3), @7=(@1*v_0), @8=(@6*@7), @9=(@1*dv_0), @10=9.81, @11=(@3+(@2*@4)), @12=((v_1*@8)-((@5*@9)+(@10*@11))), @13=(@12-(v_1*@8)), @14=((@2*@3)-@4), @15=((@10*@14)-(@6*@9)), @16=(@14*v_0), @17=(@11*v_0), @18=(@17*@16), @19=(@5*@7), @20=((@16*@19)+(@17*@8)), @21=(@14*dv_0), @22=(@11*dv_0), @23=((v_1*@16)+@22), @24=(@21-(v_1*@17)), @25=-0.1197, @26=cos(q_2), @27=sin(q_2), @28=(@8+(@25*@17)), @29=0.425, @30=(@19+((@25*@16)-(@29*v_1))), @31=((@27*@28)-(@26*@30)), @32=(@12-((@25*@24)-(@29*dv_1))), @33=((@15-(v_1*@19))-(@25*@23)), @34=((v_2*@31)+((@27*@32)+(@26*@33))), @35=((@26*@17)+(@27*@16)), @36=(@29*@17), @37=(v_2+v_1), @38=(@34-((@35*@36)+(@37*@31))), @39=((@27*@30)+(@26*@28)), @40=((v_2*@39)+((@26*@32)-(@27*@33))), @41=((@26*@16)-(@27*@17)), @42=(@40-((@37*@39)+(@41*@36))), @43=((@26*@38)-(@27*@42)), @44=(@29*@23), @45=(@44+((@41*@31)-(@35*@39))), @46=((@26*@42)+(@27*@38)), @47=(sq(@37)+sq(@41)), @48=(@

In [42]:
tau = joint_regressor(model._q, model._v, model._dv) @ parameters

inverse_dynamics = cs.Function("inverse_dynamics",
                                [model._q, model._v, model._dv, parameters],
                                [tau],
                                ["q", "v", "dv", "parameters"],
                                ["tau"],)

In [65]:
import numpy as np

In [69]:
inertia = cs.SX.zeros(nv, nv)
unit_vectors = np.eye(nv)
for i in range(nv):
    unit_vector = unit_vectors[i,:]
    inertia[:, i] = joint_regressor(model._q, np.zeros(nv), unit_vector) @ parameters
    
inertia_matrix  = cs.Function("inertia_matrix",
                                [model._q, parameters],
                                [inertia],
                                ["q", "parameters"],
                                ["inertia_matrix"],)

In [70]:
inertia_matrix

Function(inertia_matrix:(q[6],parameters[60])->(inertia_matrix[6x6]) SXFunction)

In [73]:
bias = joint_regressor(model._q, model._v, np.zeros(nv)) @ parameters

bias_term = cs.Function("bias_force",
                                [model._q, model._v, parameters],
                                [bias],
                                ["q", "v", "parameters"],
                                ["bias_force"],)

In [74]:
bias_term

Function(bias_force:(q[6],v[6],parameters[60])->(bias_force[6]) SXFunction)

In [78]:
ddq = cs.solve(inertia, model._u - bias)

forward_dynamics = cs.Function("forward_dynamics",
                                [model._q, model._v, model._u, parameters],
                                [ddq],
                                ["q", "v", "u", "parameters"],
                                ["ddq"],)